# ACO (Ant Colony Optimization)

En esta libreta se presenta una implementacion del algoritmo Ant Colony Optimization (ACO)

Primeramente se importan algunas librerias requeridas para poder manejar arreglos, obtener números aleatorios, etc.

In [1]:
import numpy as np

## Implementacion de Funciones

Ahora se implementan algunas funciones que serán de utilidad para la implementación del ACO.

In [76]:
def calcular_p(grafo, feromonas):
    """
    Calcular la probabilidad de ir a un nodo dadas las feromonas
    - input: 
        - grafo
        - feromonas: matriz con las feromonas
    - output: probabilidad de cada nodo
    """
    # Crear matriz de probabilidades
    p = np.zeros_like(feromonas, dtype=float)
    # Para cada nodo
    for i in range(len(feromonas)):
        disponibles = np.where(grafo[i])[0] # Obtener nodos que puede visitar la hormiga
        suma = np.sum(feromonas[i][disponibles])  # Suma de los valores de las feromonas para disponibles
        # iterar sobre nodos disponibles desde el nodo i
        for j in disponibles:
            p[i,j] = feromonas[i,j] / suma  # Asignar a cada nodo el valor de su fermona entre la suma de feromonas
    return p

def remover_loops(camino):
    """
    Funcion para quitar loops de un camino
    input: camino
    output: camino sin loops
    """
    # Para cada elemento en camino
    for i in range(len(camino)):
        # Comparar contra los otros elementos del camino
        for j in range(i+1, len(camino))[::-1]:
            if camino[i] == camino[j]:
                # Elimnar los numeros entre los repetidos
                camino[i:j] = []
                break
    return camino

def construir_sol(grafo, inicio, destino, p):
    """
    Esta funcion construye un camino desde el nodo inicial hasta el final y evalua el costo
    - input: 
        - adj: matriz de adjacencia del grafo
        - inicio: nodo de inicio
        - destino: nodo destino
        - p: matriz con las probabilidades
    - output:
        - camino recorrido
        - costo del camino
    """
    # Se puede hacer que la hormiga ni visite los nodos ya visitados evitando asi los loops
    camino = []
    actual = inicio  # El nodo actual es el nodo de inicio
    camino = [] # crear camino
    camino.append(inicio)
    while actual != destino:
        disponibles = np.where(grafo[actual])[0]  # Nodos a los que puede ir la hormiga desde el actual
        actual = np.random.choice(disponibles, p=p[actual][disponibles])  # Elegir siguiente nodo
        camino.append(actual)
    return remover_loops(camino)  # Eliminar loops del camino

def actualizar_feromonas(grafo, feromonas):
    """
    Actualizar los valores de las feromonas
    input:
        - grafo
        - feromonas: matriz con las feromonas
    output: feromonas actualizadas
    """
    feromonas *= (1 - rho)  # Evaporar feromonas
    # Matriz con los dtau para actualizar las feromonas
    dtau = np.zeros_like(feromonas, dtype=float)
    for c in caminos:
        # Matriz temporal para ir guardando los Q/f(x) y despues sumar los de todos los caminos
        temp = np.zeros_like(feromonas, dtype=float)
        costo = 0  # Costo del camino
        for i in range(1, len(c)):
            # Feromona que si se actualizara
            temp[c[i-1], c[i]] = 1
            temp[c[i], c[i-1]] = 1
            costo += grafo[c[i-1], c[i]]
        temp *= Q/costo
        dtau += temp

    # Actualizar feromonas
    feromonas += dtau
    return feromonas
        

In [235]:
grafo = np.array([[0, 0, 0, 1, 0, 1, 0],
                  [0, 0, 1, 1, 0, 0, 0],
                  [0, 1, 0, 1, 1, 0, 0],
                  [1, 1, 1, 0, 1, 1, 0],
                  [0, 0, 1, 1, 0, 0, 1],
                  [1, 0, 0, 1, 0, 0, 1],
                  [0, 0, 0, 0, 1, 1, 0]])

n = 50  # Numero de hormigas
Q = 0.5
rho = 0.25  # Evaporacion de feromonas (0,1) mayor rho, se evaporan mas rapido
iteraciones = 50


inicio = 0  # De 0 a (numero de nodos - 1)
destino = 4

In [237]:
# Inicializar feromonas
feromonas = np.ones_like(grafo, dtype=float) # hacer las feromonas de la diagonal 0?

for i in range(iteraciones):
    # Calcular probabilidades
    p = calcular_p(grafo, feromonas)
    # Construir n caminos
    caminos = []
    for i in range(n):
        caminos.append(construir_sol(grafo, inicio, destino, p))
    actualizar_feromonas(grafo, feromonas)
    
camino_optimo = construir_sol(grafo, inicio, destino, p)
# Calcular costo del camino
costo_minimo = 0
for i in range(1, len(camino_optimo)):
    costo_minimo += grafo[camino_optimo[i-1], camino_optimo[i]]
print(f"El camino mas corto es {camino_optimo} con un costo de {costo_minimo}")

El camino mas corto es [0, 3, 4] con un costo de 2
